In [9]:

import pandas as pd
import xlsxwriter

def create_tax_system_v26(filename='房地稅務全方位系統_V26(透明詳解版).xlsx'):
    """
    V26.0 透明詳解版：
    1. 【Step 2 雙算式】土地與建物下方各自新增獨立算式列。
       - 建物：淨額 × 比例
       - 土地：淨額 - 建物 (明確顯示倒扣邏輯)
    2. 【Step 3 明細化】核實認定的算式字串完整展開成本與費用數值，不再使用 "..."。
    3. 【核心運算】維持 V25 的 Excel 高精度運算，確保金額準確。
    """

    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    workbook = writer.book
    FONT_NAME = 'Microsoft JhengHei'

    sheet_calc = workbook.add_worksheet('房地稅務試算')
    sheet_ref = workbook.add_worksheet('稅率參照表')

    sheet_calc.hide_gridlines(2)

    # ==========================================
    # 樣式定義 (Styles)
    # ==========================================
    
    fmt_main_title = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 18,
        'align': 'left', 'valign': 'vcenter', 'font_color': '#203764'
    })

    fmt_step_header = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 13, 'align': 'left', 'valign': 'vcenter',
        'bg_color': '#4472C4', 'font_color': 'white', 'border': 1
    })
    
    fmt_input_label = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 11, 'align': 'left', 'valign': 'vcenter',
        'bg_color': '#FFF2CC', 'border': 1, 'border_color': '#BF9000'
    })
    
    fmt_input_val = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 12, 'align': 'right', 'valign': 'vcenter',
        'num_format': '#,##0', 'bg_color': '#FFFFE0', 'border': 1, 'border_color': '#BF9000', 'font_color': 'blue'
    })
    
    fmt_input_rate_free = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 12, 'align': 'center', 'valign': 'vcenter',
        'bg_color': '#FFFFE0', 'border': 1, 'border_color': '#BF9000', 'font_color': 'blue'
    })
    
    fmt_user_fill = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 12, 'align': 'center', 'valign': 'vcenter',
        'bg_color': '#FFFF99', 'border': 2, 'border_color': '#FF0000', 'font_color': '#FF0000'
    })

    fmt_row_header = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'align': 'center', 'valign': 'vcenter',
        'bg_color': '#E7E6E6', 'font_color': '#333333', 'border': 1
    })
    
    fmt_res_green = workbook.add_format({ 
        'font_name': FONT_NAME, 'bold': True, 'font_size': 12, 'align': 'right', 'valign': 'vcenter',
        'num_format': '#,##0', 'bg_color': '#E2EFDA', 'border': 1, 'border_color': '#548235', 'font_color': '#385723'
    })
    
    fmt_res_blue = workbook.add_format({ 
        'font_name': FONT_NAME, 'bold': True, 'font_size': 12, 'align': 'right', 'valign': 'vcenter',
        'num_format': '#,##0', 'bg_color': '#DDEBF7', 'border': 1, 'border_color': '#9BC2E6', 'font_color': '#1F4E78'
    })
    
    fmt_res_ratio = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 12, 'align': 'center', 'valign': 'vcenter',
        'num_format': '0.00%', 'bg_color': '#E2EFDA', 'border': 1, 'border_color': '#548235', 'font_color': '#385723'
    })

    # 算式顯示區 (稍微縮小字體以容納長算式)
    fmt_formula_display = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 9, 'align': 'right', 'valign': 'center',
        'font_color': '#595959', 'bg_color': '#F2F2F2', 'border': 1, 'border_color': '#D9D9D9'
    })
    
    fmt_unit = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 10, 'align': 'left', 'valign': 'center',
        'font_color': 'gray'
    })

    fmt_exp_header = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'align': 'center', 'bg_color': '#FFD966', 'border': 1
    })
    fmt_exp_item = workbook.add_format({
        'font_name': FONT_NAME, 'align': 'left', 'bg_color': 'white', 'border': 1
    })
    fmt_exp_val = workbook.add_format({
        'font_name': FONT_NAME, 'align': 'right', 'num_format': '#,##0', 'bg_color': '#FFFFE0', 'border': 1
    })

    fmt_tax_res = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 13, 'align': 'right',
        'num_format': '"NT$ "#,##0', 'bg_color': '#FCE4D6', 'font_color': '#C00000', 'border': 1
    })

    fmt_guide_title = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 12, 'bold': True, 'color': 'white',
        'bg_color': '#548235', 'align': 'center', 'valign': 'vcenter', 'border': 1
    })
    fmt_guide_body = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 10, 'color': '#333333',
        'text_wrap': True, 'valign': 'top', 'align': 'left',
        'bg_color': 'white', 'border': 1, 'border_color': '#E2EFDA'
    })

    # ==========================================
    # 佈局
    # ==========================================
    sheet_calc.set_column('A:A', 2)
    sheet_calc.set_column('B:B', 22)
    sheet_calc.set_column('C:C', 20)
    sheet_calc.set_column('D:D', 28)
    sheet_calc.set_column('E:E', 2)
    sheet_calc.set_column('F:G', 15)
    sheet_calc.set_column('H:H', 2)
    sheet_calc.set_column('I:K', 16)

    # ==========================================
    # 內容建置
    # ==========================================
    sheet_calc.merge_range('B1:G1', '房地稅務全方位系統 (V26 透明詳解版)', fmt_main_title)

    # ---------------------------------------------------
    # Step 1: 基礎資料
    # ---------------------------------------------------
    r = 2
    sheet_calc.merge_range(r, 1, r, 3, " 步驟 1：基礎資料輸入", fmt_step_header)
    
    input_list = [
        ("土地公告現值 (L)", 0),
        ("建物評定現值 (B)", 0),
        ("成交總價 (T)", 0),
        ("車位價格 (C)", 0),
    ]
    
    r += 1
    for label, val in input_list:
        sheet_calc.write(r, 1, label, fmt_input_label)
        sheet_calc.write(r, 2, val, fmt_input_val)
        sheet_calc.write(r, 3, "元", fmt_unit)
        r += 1
    
    L, B, T, C = "C4", "C5", "C6", "C7"
    
    # 房地比 (精確值)
    sheet_calc.write(r, 1, "★ 房地比 (精確值)", fmt_row_header)
    sheet_calc.write_formula(r, 2, f'=IF(({L}+{B})=0,0, {B}/({L}+{B}))', fmt_res_ratio)
    sheet_calc.write(r, 3, "顯示2位，運算採完整精度", fmt_unit)
    Ratio = f"C{r+1}"
    
    # ---------------------------------------------------
    # 右側費用表
    # ---------------------------------------------------
    exp_start_row = 2
    sheet_calc.merge_range(exp_start_row, 5, exp_start_row, 6, "【成本費用明細表】", fmt_exp_header)
    
    expenses = [
        ("取得-契稅", 0), ("取得-印花稅", 0), ("取得-代書費", 0),
        ("取得-仲介費", 0), ("取得-履保費", 0), ("出售-土增稅", 0), 
        ("出售-代書費", 0), ("出售-仲介費", 0), ("改良-裝潢費", 0), 
        ("其他必要費用", 0)
    ]
    
    er = exp_start_row + 1
    for name, val in expenses:
        sheet_calc.write(er, 5, name, fmt_exp_item)
        sheet_calc.write(er, 6, val, fmt_exp_val)
        er += 1
    
    sheet_calc.write(er, 5, "費用總計", fmt_exp_header)
    sheet_calc.write_formula(er, 6, f'=SUM(G{exp_start_row+2}:G{er})', fmt_exp_val)
    Exp_Total = f"G{er+1}"

    # ---------------------------------------------------
    # Step 2: 房地拆價 (算式展開)
    # ---------------------------------------------------
    r += 2
    sheet_calc.merge_range(r, 1, r, 3, " 步驟 2：房地拆價 (完整算式)", fmt_step_header)
    
    r += 1
    sheet_calc.write(r, 1, "項目", fmt_row_header)
    sheet_calc.write(r, 2, "一般個人 (未稅)", fmt_row_header)
    sheet_calc.write(r, 3, "法人/公司 (含稅)", fmt_row_header)
    
    Net_Txt = f'TEXT({T}-{C}, "#,##0")'

    # --- 建物 ---
    r += 1
    sheet_calc.write(r, 1, "建物價格", fmt_input_label)
    sheet_calc.write_formula(r, 2, f"=IF({T}=0,0, ({T}-{C}) * {Ratio})", fmt_res_green)
    Cell_B_Gen = f"C{r+1}"
    sheet_calc.write_formula(r, 3, f"=IF({T}=0,0, ({T}-{C}) * (({B}*1.05)/({L}+{B}*1.05)))", fmt_res_blue)
    Cell_B_Corp = f"D{r+1}"
    
    # [新增] 建物算式列
    r += 1
    f_cal_B_Gen = f'="算式：" & {Net_Txt} & " × " & TEXT({Ratio}, "0.00%") & " (比例)"'
    sheet_calc.write_formula(r, 2, f_cal_B_Gen, fmt_formula_display)
    f_cal_B_Corp = f'="算式：" & {Net_Txt} & " × " & TEXT(({B}*1.05)/({L}+{B}*1.05), "0.00%") & " (比例)"'
    sheet_calc.write_formula(r, 3, f_cal_B_Corp, fmt_formula_display)
    
    # --- 土地 ---
    r += 1
    sheet_calc.write(r, 1, "土地價格", fmt_input_label)
    sheet_calc.write_formula(r, 2, f"=IF({T}=0,0, ({T}-{C}) - {Cell_B_Gen})", fmt_res_green)
    sheet_calc.write_formula(r, 3, f"=IF({T}=0,0, ({T}-{C}) - {Cell_B_Corp})", fmt_res_blue)
    
    # [新增] 土地算式列 (倒扣法文字說明)
    r += 1
    f_cal_L_Gen = f'="算式：" & {Net_Txt} & " - " & TEXT({Cell_B_Gen}, "#,##0") & " (建物)"'
    sheet_calc.write_formula(r, 2, f_cal_L_Gen, fmt_formula_display)
    f_cal_L_Corp = f'="算式：" & {Net_Txt} & " - " & TEXT({Cell_B_Corp}, "#,##0") & " (建物)"'
    sheet_calc.write_formula(r, 3, f_cal_L_Corp, fmt_formula_display)

    # ---------------------------------------------------
    # Step 3: 所得申報模式
    # ---------------------------------------------------
    r += 2
    sheet_calc.merge_range(r, 1, r, 3, " 步驟 3：所得計算 (三選一)", fmt_step_header)
    
    # Mode A (核實)
    r += 1
    sheet_calc.merge_range(r, 1, r, 3, "A. 核實認定 (售價 - 成本 - 費用)", workbook.add_format({'bold': True, 'bg_color': '#D9E1F2', 'border': 1}))
    r += 1
    sheet_calc.write(r, 1, "原始取得成本", fmt_input_label)
    sheet_calc.write(r, 2, 0, fmt_input_val)
    Cost_Orig = f"C{r+1}"
    sheet_calc.write(r, 3, "費用自動扣除", fmt_unit)
    
    r += 1
    sheet_calc.write(r, 1, "★ 核實所得額", fmt_row_header)
    sheet_calc.write_formula(r, 2, f'=MAX(0, ({T} - {Cost_Orig} - {Exp_Total}) * {Ratio})', fmt_res_green)
    Income_A = f"C{r+1}"
    
    # Mode A 算式 (透明展開：成本與費用數值)
    r += 1
    sheet_calc.write(r, 1, "驗算算式", fmt_row_header)
    # 這裡將 Exp_Total 也放進文字字串
    f_cal_A_Full = f'="算式：( " & TEXT({T},"#,##0") & " - " & TEXT({Cost_Orig},"#,##0") & " - " & TEXT({Exp_Total},"#,##0") & " (費用) ) × " & TEXT({Ratio},"0.00%")'
    sheet_calc.merge_range(r, 2, r, 3, f_cal_A_Full, fmt_formula_display)

    # Mode B (標準)
    r += 1
    sheet_calc.merge_range(r, 1, r, 3, "B. 標準設算 (房屋現值 × 標準率)", workbook.add_format({'bold': True, 'bg_color': '#D9E1F2', 'border': 1}))
    r += 1
    sheet_calc.write(r, 1, "輸入標準率 (如42)", fmt_input_label)
    sheet_calc.write(r, 2, "", fmt_user_fill)
    Rate_Std_Input = f"C{r+1}"
    sheet_calc.write(r, 3, "輸入 42 或 0.42 皆可", fmt_unit)
    
    r += 1
    sheet_calc.write(r, 1, "★ 標準設算所得", fmt_row_header)
    Real_Rate_B = f"IF({Rate_Std_Input}<1, {Rate_Std_Input}, {Rate_Std_Input}/100)"
    sheet_calc.write_formula(r, 2, f'=IF(ISNUMBER({Rate_Std_Input}), ROUND({B} * {Real_Rate_B}, 0), 0)', fmt_res_green)
    Income_B = f"C{r+1}"
    
    r += 1
    sheet_calc.write(r, 1, "驗算算式", fmt_row_header)
    Txt_Rate_B = f"IF({Rate_Std_Input}<1, {Rate_Std_Input}*100, {Rate_Std_Input})"
    f_cal_B = f'="算式： " & TEXT({B},"#,##0") & " × " & {Txt_Rate_B} & "%"'
    sheet_calc.merge_range(r, 2, r, 3, f_cal_B, fmt_formula_display)

    # Mode C (豪宅)
    r += 1
    sheet_calc.merge_range(r, 1, r, 3, "C. 豪宅設算 (總價 × 房地比 × 20%)", workbook.add_format({'bold': True, 'bg_color': '#D9E1F2', 'border': 1}))
    r += 1
    sheet_calc.write(r, 1, "豪宅歸屬率", fmt_input_label)
    sheet_calc.write(r, 2, 20, fmt_input_rate_free)
    Rate_High_Input = f"C{r+1}"
    sheet_calc.write(r, 3, "% (法定固定)", fmt_unit)
    
    r += 1
    sheet_calc.write(r, 1, "★ 豪宅設算所得", fmt_row_header)
    Real_Rate_C = f"IF({Rate_High_Input}<1, {Rate_High_Input}, {Rate_High_Input}/100)"
    sheet_calc.write_formula(r, 2, f'=ROUND({T} * {Ratio} * {Real_Rate_C}, 0)', fmt_res_green)
    Income_C = f"C{r+1}"
    
    r += 1
    sheet_calc.write(r, 1, "驗算算式", fmt_row_header)
    Txt_Rate_C = f"IF({Rate_High_Input}<1, {Rate_High_Input}*100, {Rate_High_Input})"
    f_cal_C = f'="算式： " & TEXT({T},"#,##0") & " × " & TEXT({Ratio},"0.00%") & " × " & {Txt_Rate_C} & "%"'
    sheet_calc.merge_range(r, 2, r, 3, f_cal_C, fmt_formula_display)

    # ---------------------------------------------------
    # Step 4: 最終稅額
    # ---------------------------------------------------
    r += 2
    sheet_calc.merge_range(r, 1, r, 3, " 步驟 4：綜所稅額試算", fmt_step_header)
    
    r += 1
    sheet_calc.write(r, 1, "申報所得額", fmt_input_label)
    sheet_calc.write(r, 2, 0, fmt_input_val)
    sheet_calc.write(r, 3, "← 填入上方結果", fmt_unit)
    Final_Income = f"C{r+1}"
    
    r += 1
    sheet_calc.write(r, 1, "適用稅率 (如 30)", fmt_input_label)
    sheet_calc.write(r, 2, 20, fmt_input_rate_free)
    Final_Rate_Input = f"C{r+1}"
    sheet_calc.write(r, 3, "查右表填入", fmt_unit)
    
    r += 1
    sheet_calc.write(r, 1, "累進差額", fmt_input_label)
    sheet_calc.write(r, 2, 147700, fmt_input_val)
    Final_Diff = f"C{r+1}"
    
    r += 1
    sheet_calc.write(r, 1, "預估應納稅額", fmt_row_header)
    Real_Final_Rate = f"IF({Final_Rate_Input}<1, {Final_Rate_Input}, {Final_Rate_Input}/100)"
    sheet_calc.write_formula(r, 2, f'=ROUND({Final_Income}*{Real_Final_Rate}-{Final_Diff}, 0)', fmt_tax_res)
    
    r += 1
    sheet_calc.write(r, 1, "驗算算式", fmt_row_header)
    Txt_Final_Rate = f"IF({Final_Rate_Input}<1, {Final_Rate_Input}*100, {Final_Rate_Input})"
    f_cal_Final = f'="算式： " & TEXT({Final_Income},"#,##0") & " × " & {Txt_Final_Rate} & "% - " & TEXT({Final_Diff},"#,##0")'
    sheet_calc.merge_range(r, 2, r, 3, f_cal_Final, fmt_formula_display)

    # ---------------------------------------------------
    # 右側：指南
    # ---------------------------------------------------
    sheet_calc.merge_range('I2:K2', '系統使用指南與計算邏輯', fmt_guide_title)
    
    full_guide_text = (
        "【步驟1】填寫黃色欄位：房屋/土地現值、成交總價。\n\n"
        "【步驟2】系統會自動進行「房地拆價」，並分別列出土地與建物的算式。\n"
        "   (註：為確保土地與建物相加嚴格等於總價，土地價格採用「倒扣法」計算，即 [總價-建物]。)\n\n"
        "【步驟3】選擇所得申報方式：\n"
        " ● A.核實認定：若您有原始買價，請在右側「費用明細表」填寫支出，算式中會顯示總費用金額。\n"
        " ● B.標準設算：若無成本證明，輸入當地標準率(如42)。\n"
        " ● C.豪宅設算：高總價案件專用(固定稅率)。\n\n"
        "【步驟4】將算出的所得額填入下方，即可試算綜所稅額。\n"
        "--------------------------------------\n"
        "1. 拆算核心邏輯：\n"
        "   先將 [成交總價] 扣除 [車位價格] 得到房地淨額，再依下列比例拆分。\n"
        "   ★ 本表採用精確比例運算 (Excel 完整精度)，確保金額分配準確。\n\n"
        "2. 兩種情境的比例計算方式：\n"
        "   ● 一般個人 (未稅)：\n"
        "     → 建物比例 = 建物現值 ÷ 總現值\n"
        "     → 建物價格 = 房地淨額 × 建物比例\n"
        "     → 土地價格 = 房地淨額 - 建物價格 (確保總和正確)\n\n"
        "   ● 法人/公司 (含稅)：\n"
        "     因法人出售建物需開立含 5% 營業稅發票，故建物價值需加權。\n"
        "     → 建物比例公式：(建物現值 × 1.05) ÷ (土地現值 + 建物現值 × 1.05)\n\n"
        "3. 房地比與財交稅：\n"
        "   房地比格式顯示為百分比 (如 30.69%)，方便直接填寫申報書。\n\n"
        "4. ★ 高總價/豪宅特殊規範：\n"
        "   公式：[成交總價] × 房地比(0.xx) × 歸屬率"
    )
    sheet_calc.merge_range('I3:K42', full_guide_text, fmt_guide_body)

    sheet_calc.merge_range(er+2, 5, er+2, 6, "113年度稅率級距", fmt_exp_header)
    tax_brackets = [
        ("0~59萬", "5%", "0"),
        ("59~133萬", "12%", "41,300"),
        ("133~266萬", "20%", "147,700"),
        ("266~498萬", "30%", "413,700"),
        ("498萬以上", "40%", "911,700")
    ]
    tr = er + 3
    for label, rate, diff in tax_brackets:
        sheet_calc.write(tr, 5, label, fmt_exp_item)
        sheet_calc.write(tr, 6, f"{rate} (-{diff})", fmt_exp_item)
        tr += 1

    writer.close()
    print(f"檔案已生成：{filename}")

if __name__ == "__main__":
    create_tax_system_v26()



檔案已生成：房地稅務全方位系統_V26(透明詳解版).xlsx
